In [ ]:
!pip install efficientnet_pytorch

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import torch
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import torchvision
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchvision import models, transforms
from efficientnet_pytorch import EfficientNet
import copy
import tqdm
import PIL
from PIL import Image
import zipfile

from sklearn.metrics import confusion_matrix

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train_data_path = "../input/dogs-vs-cats-redux-kernels-edition/train.zip"
test_data_path = "../input/dogs-vs-cats-redux-kernels-edition/test.zip"

with zipfile.ZipFile(train_data_path, 'r') as zip_ref:
    zip_ref.extractall(".")
    
with zipfile.ZipFile(test_data_path, 'r') as zip_ref:
    zip_ref.extractall(".")

In [ ]:
import glob

train_dir = "./train"
test_dir = "./test"
train_list = glob.glob(os.path.join(train_dir,'*.jpg'))
test_list = glob.glob(os.path.join(test_dir, '*.jpg'))

In [ ]:
from sklearn.model_selection import train_test_split
train_list, val_list = train_test_split(train_list, test_size=0.2)

In [ ]:
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

train_transforms =  transforms.Compose([
        transforms.Resize((448, 448)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ])

val_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ])


test_transforms = transforms.Compose([   
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
    ])

In [ ]:
class dataset(torch.utils.data.Dataset):
    def __init__(self,file_list,transform=None):
        self.file_list = file_list
        self.transform = transform
        
    def __len__(self):
        self.filelength = len(self.file_list)
        return self.filelength
    
    #load an one of images
    def __getitem__(self,idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path)
        img_transformed = self.transform(img)
        
        label = img_path.split('/')[-1].split('.')[0]
        if label == 'dog':
            label=1
        elif label == 'cat':
            label=0
            
        return img_transformed,label

In [ ]:
train_data = dataset(train_list, transform=train_transforms)
test_data = dataset(test_list, transform=test_transforms)
val_data = dataset(val_list, transform=test_transforms)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size=32, shuffle=True )
test_loader = torch.utils.data.DataLoader(dataset = test_data, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset = val_data, batch_size=32, shuffle=True)

In [ ]:
# samples, labels = iter(train_loader).next()
# fig = plt.figure(figsize=(24, 16))
# fig.tight_layout()
# ad = {0:'cat', 1:'dog'}
# for num, sample in enumerate(samples[:24]):
#     plt.subplot(4,6,num+1)
#     plt.title(ad[labels[num].item()])
#     plt.axis('off')
#     sample = sample.cpu().numpy()
#     plt.imshow(np.transpose(sample, (1,2,0)))

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(42)
if device =='cuda':
    torch.cuda.manual_seed_all(42)

# **CNN**

In [ ]:
# class Cnn(nn.Module):
#     def __init__(self):
#         super(Cnn,self).__init__()
        
#         self.layer1 = nn.Sequential(
#             nn.Conv2d(3,16,kernel_size=3, padding=0,stride=2),
#             nn.BatchNorm2d(16),
#             nn.ReLU(),
#             nn.MaxPool2d(2)
#         )
        
#         self.layer2 = nn.Sequential(
#             nn.Conv2d(16,32, kernel_size=3, padding=0, stride=2),
#             nn.BatchNorm2d(32),
#             nn.ReLU(),
#             nn.MaxPool2d(2)
#             )
        
#         self.layer3 = nn.Sequential(
#             nn.Conv2d(32,64, kernel_size=3, padding=0, stride=2),
#             nn.BatchNorm2d(64),
#             nn.ReLU(),
#             nn.MaxPool2d(2)
#         )
        
        
#         self.fc1 = nn.Linear(3*3*64,10)
#         self.dropout = nn.Dropout(0.5)
#         self.fc2 = nn.Linear(10,2)
#         self.relu = nn.ReLU()
        
        
#     def forward(self,x):
#         out = self.layer1(x)
#         out = self.layer2(out)
#         out = self.layer3(out)
#         out = out.view(out.size(0),-1)
#         out = self.relu(self.fc1(out))
#         out = self.fc2(out)
#         return out

In [ ]:
# model = Cnn().to(device)
# model.train()

In [ ]:
# optimizer = optim.Adam(params = model.parameters(),lr=0.001)
# criterion = nn.CrossEntropyLoss()

# **VGG16/19**

In [ ]:
model = models.vgg19(pretrained=True)
# model

In [ ]:
model.classifier[6] = nn.Linear(in_features=4096, out_features=1)

update_params = []
params_names = ['classifier.6.weight', 'classifier.6.bias']

for name, param in model.named_parameters():
    if name in params_names:
        param.requires_grad = True
        update_params.append(param)
    else:
        param.requires_grad = False
        
model = model.to(device)
sigmo = nn.Sigmoid()

In [ ]:
# criterion = nn.CrossEntropyLoss()
# criterion = nn.BCEWithLogitsLoss(reduction = 'mean').to(device)
criterion = nn.BCELoss(reduction = 'mean').to(device)
optimizer = optim.SGD(params=update_params, lr=0.001, momentum=0.9)

# **InceptionV3**

In [ ]:
# model = models.inception_v3(pretrained=True, aux_logits=False)
# model

In [ ]:
# model = models.inception_v3(pretrained=True, aux_logits=False)
# model.fc = nn.Linear(in_features=2048, out_features=2)

# update_params = []
# params_names = ['fc.weight', 'fc.bias']

# for name, param in model.named_parameters():
#     if name in params_names:
#         param.requires_grad = True
#         update_params.append(param)
#     else:
#         param.requires_grad = False
        
# model = model.to(device)

In [ ]:
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(params=update_params, lr=0.001, momentum=0.9)

# **EffNet**

In [ ]:
# model = EfficientNet.from_pretrained("efficientnet-b5")
# model

In [ ]:
# model._fc = nn.Linear(in_features=2048, out_features=2)
# update_params = []
# params_names = ['_fc.weight', '_fc.bias']

# for name, param in model.named_parameters():
#     if name in params_names:
#         param.requires_grad = True
#         update_params.append(param)
#     else:
#         param.requires_grad = False
        
# model = model.to(device)

In [ ]:
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(params=update_params, lr=0.001, momentum=0.9)

In [ ]:
epochs = 6

train_loss_list = []
train_acc_list = []
val_loss_list = []
val_acc_list = []

best_loss = 1e10
weights_path = "."

for epoch in range(epochs):
    epoch_loss = 0
    epoch_accuracy = 0
#     print("#batches ", len(train_loader))
    for batch_idx, (data, label) in enumerate(train_loader):
#         if (batch_idx+1) % 100 == 0:
#             print(batch_idx+1)
        data = data.to(device)
        label = label.to(device)
        
        output = sigmo(model(data))
        loss = criterion(output, label.view(-1, 1).float())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
#         acc = ((output.argmax(dim=1) == label).float().mean())
        acc = (((torch.flatten(output) > 0.5).int() == label).float().mean())
        epoch_accuracy += acc/len(train_loader)
        epoch_loss += loss/len(train_loader)
        
    print('Epoch : {}, train accuracy : {}, train loss : {}'.format(epoch+1, epoch_accuracy,epoch_loss))
    train_loss_list.append(epoch_loss)
    train_acc_list.append(epoch_accuracy)
    
    with torch.no_grad():
        epoch_val_accuracy=0
        epoch_val_loss =0
        for data, label in val_loader:
            data = data.to(device)
            label = label.to(device)
            
            val_output = sigmo(model(data))
            val_loss = criterion(val_output,label.view(-1, 1).float())
            
            
#             acc = ((val_output.argmax(dim=1) == label).float().mean())
            acc = (((torch.flatten(val_output) > 0.5).int() == label).float().mean())
            epoch_val_accuracy += acc/ len(val_loader)
            epoch_val_loss += val_loss/ len(val_loader)
            
        print('Epoch : {}, val_accuracy : {}, val_loss : {}'.format(epoch+1, epoch_val_accuracy,epoch_val_loss))
        val_loss_list.append(epoch_val_loss)
        val_acc_list.append(epoch_val_accuracy)
        if epoch_val_loss < best_loss:
            torch.save(model.state_dict(), os.path.join(weights_path, "model.pt"),)

In [ ]:
plt.clf()
x_axis = list(range(epochs))
plt.plot(x_axis, train_loss_list, label="Train")
plt.plot(x_axis, val_loss_list, label="Validation")
plt.xlabel("#epoch")
plt.ylabel('loss')
plt.title('Loss')
plt.legend()
plt.show()

plt.clf()
x_axis = list(range(epochs))
plt.plot(x_axis, train_acc_list, label="Train")
plt.plot(x_axis, val_acc_list, label="Validation")
plt.xlabel("#epoch")
plt.ylabel('Accuracy')
plt.title('accuracy')
plt.legend()
plt.show()

In [ ]:
predictions = []
true_labels = []
wrong_answers = []
epoch_val_loss = 0
epoch_val_accuracy = 0
for data, label in val_loader:
    data = data.to(device)
    label = label.to(device)

    val_output = sigmo(model(data))
    val_output = torch.flatten(val_output)
    for i in range(len(val_output)):
        predictions.append((val_output[i] > 0.5).int().item())
        true_labels.append(label[i].item())
        if predictions[-1] != true_labels[-1]:
            wrong_answers.append((data[i], predictions[-1]))

In [ ]:
class UnNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        for t, m, s in zip(tensor, self.mean, self.std):
            t.mul_(s).add_(m)
            # The normalize code -> t.sub_(m).div_(s)
        return tensor

In [ ]:
print(true_labels[:20])
print(predictions[:20])
unorm = UnNormalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))

fig = plt.figure(figsize=(24, 16))
fig.tight_layout()
ad = {0:'cat', 1:'dog'}
for num, (sample, label) in enumerate(wrong_answers[:36]):
    plt.subplot(6,6,num+1)
    plt.title(ad[label])
    plt.axis('off')
    
    sample = unorm(sample)
    sample = sample.detach().cpu().numpy()
    plt.imshow(np.transpose(sample, (1,2,0)))

In [ ]:
conf_matrix = confusion_matrix(true_labels, predictions)

fig, ax = plt.subplots(figsize=(7.5, 7.5))
ax.matshow(conf_matrix, cmap=plt.cm.Blues, alpha=0.3)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')
 
plt.xlabel('Predictions', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('Confusion Matrix', fontsize=18)
plt.show()

In [ ]:
model.load_state_dict(torch.load(weights_path + "/model.pt", map_location=device))
dog_probs = []
model.eval()
with torch.no_grad():
    for data, fileid in test_loader:
        data = data.to(device)
        preds = sigmo(model(data))
#         preds_list = F.softmax(preds, dim=1)[:, 1].tolist()
#         preds_list = (torch.flatten(preds) > 0.5).detach().float().cpu() * 0.97 + 0.01
        preds_list = torch.flatten(preds).detach().cpu().numpy()
        dog_probs += list(zip(list(fileid), preds_list))

In [ ]:
dog_probs.sort(key = lambda x : int(x[0]))
idx = list(map(lambda x: x[0],dog_probs))
prob = list(map(lambda x: x[1],dog_probs))
submission = pd.DataFrame({'id':idx,'label':prob})
submission

In [ ]:
submission.to_csv('last.csv',index=False)